In [3]:
# load env ------------------------------------------------------------------------
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "false"


# debug ------------------------------------------------------------------
from langchain.globals import set_debug, set_verbose
set_verbose(True)
set_debug(False)

# from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    ToolMessage
)
from langgraph.graph import END, StateGraph, START
from tools import get_tools_output
from agents import(
    AgentState,
    agents,
    agent_name
)
from tools import all_tools
from chat_history import save_chat_history, load_chat_history

## Define Tool Node
from langgraph.prebuilt import ToolNode
from typing import Literal

tool_node = ToolNode(all_tools)

def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "continue" in last_message.content:
        return "continue"
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "%SIjfE923hf" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    else:
        return "continue"


## Workflow Graph ------------------------------------------------------------------------
workflow = StateGraph(AgentState)

# add agent nodes
for name, value in agents.items():
    workflow.add_node(name, value['node'])
    
workflow.add_node("call_tool", tool_node)


workflow.add_conditional_edges(
    "analyst",
    router,
    {"continue": "data_collector", "call_tool": "call_tool", "__end__": END}
)

workflow.add_conditional_edges(
    "data_collector",
    router,
    {"call_tool": "call_tool", "continue": "reporter", "__end__": END}
)

workflow.add_conditional_edges(
    "reporter",
    router,
    {"continue": "data_collector", "call_tool": "call_tool", "__end__": END}
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {name:name for name in agent_name},
)

workflow.add_edge(START, "analyst")
graph = workflow.compile()


def submitUserMessage(user_input: str, user_id:str="test", keep_chat_history:bool=False, return_reference:bool=False, verbose=False) -> str:
    
    chat_history = load_chat_history(user_id=user_id) if keep_chat_history else []
    chat_history = chat_history[-8:]
    
    graph = workflow.compile()

    events = graph.stream(
        {
            "messages": [
                HumanMessage(
                    user_input
                )
            ],
            "chat_history": chat_history
        },
        # Maximum number of steps to take in the graph
        {"recursion_limit": 20},
    )
    
    if not verbose:
        events = [e for e in events]
        response = list(events[-1].values())[0]
    else:
        for e in events:
            # print(e)
            a = list(e.items())[0]
            a[1]['messages'][0].pretty_print()
        
        response = a[1]
    
    response = response["messages"][0].content
    response = response.replace("%SIjfE923hf", "")
    
    if keep_chat_history:
        save_chat_history(bot_message=response, human_message=user_input, user_id=user_id)
    
    if return_reference:
        return response, get_tools_output()
    else:
        return response

In [5]:
# import utils
submitUserMessage = utils.notify_process(submitUserMessage, sound_effect="purr")

result = submitUserMessage("วิเคราะห์ร้านอาหารไทยแถวลุมพินี เซ็นเตอร์ ลาดพร้าว", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: analyst
Tool Calls:
  duckduckgo_search (call_njniNUo8qzInXIrSG1sFZKii)
 Call ID: call_njniNUo8qzInXIrSG1sFZKii
  Args:
    query: ร้านอาหารไทยแถวลุมพินี เซ็นเตอร์ ลาดพร้าว
  restaurant_sale_projection (call_9I3ppxFcvzwdbCEluJgEpytm)
 Call ID: call_9I3ppxFcvzwdbCEluJgEpytm
  Args:
    input_dict: {'base_price': 200, 'category': 'Rice Bowl'}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


================================= Tool Message =================================
Name: duckduckgo_search

มัดรวม 26 ร้านอาหาร เซ็นทรัลลาดพร้าว ทั้งในโซนห้างและโรงแรม อาทิ 1. Sushi Seki 2.Audrey Cafe 3.Laem Charoen Seafood 4.Shabushi 5.Tops EATERY. กินอะไรดีที่ ร้านอาหาร เซ็นทรัลลาดพร้าว 2024 ย่าน ลาดพร้าว เหมือนเป็นทำเลทองของคนกรุงเทพ เพราะย่านนี้มี ร้านอาหารอร่อยๆ ... ชี้เป้าร้านอร่อย ร้านอาหาร ห้าแยกลาดพร้าว อัปเดตล่าสุด 2567 รวมร้านอาหารอร่อย ร้านอาหารใกล้ฉัน บรรยากาศดี แถมอยู่ไม่ไกลจาก bts ... คู่รัก ดินเนอร์ ร้านอาหารโรแมนติก โอกาสพิเศษ. ออกเดท ทั้งทีต้องสร้างความประทับใจ Hungry Hub ขอแนะนำ 20 ร้านบรรยากาศโรแมนติก อาทิ 1 ... 23 มิ.ย. 64. ร้านอาหาร ลาดพร้าว บรรยากาศดี อัปเดต 2024 อมรินทร์ทีวีพาไปรู้จักร้านอร่อย ๆ แถว ลาดพร้าว และไปชิลกับบรรยากาศเลิศๆ ที่ ...
================================== Ai Message ==================================
Name: analyst

%SIjfE923hf

### การวิเคราะห์ร้านอาหารไทยแถวลุมพินี เซ็นเตอร์ ลาดพร้าว

#### 1. ข้อมูลร้านอาหารในพื้นที่
- **ร้านอาหารที่มีชื่อเสีย

In [ ]:
submitUserMessage("if i it passible to sell rice 2000 bath each.", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: analyst
Tool Calls:
  restaurant_sale_projection (call_VfubgZyMobX1VE6pIN4dx6CK)
 Call ID: call_VfubgZyMobX1VE6pIN4dx6CK
  Args:
    input_dict: {'base_price': 2000, 'category': 'Rice Bowl'}
================================= Tool Message =================================
Name: restaurant_sale_projection

sale projection of Rice Bowl:
week	number of order	sale(forecast)
1	427	854,800
5	1,393	2,785,400
52	446	892,000

================================== Ai Message ==================================
Name: analyst

Selling rice at 2000 baht each is technically possible, as the sales projection indicates that you could generate significant revenue. Here are the details:

### Sales Projection for Rice Bowl at 2000 Baht Each
- **Week 1**: 
  - Number of Orders: 427
  - Total Sales: 854,800 Baht
- **Week 5**: 
  - Number of Orders: 1,393
  - Total Sales: 2,785,400 Baht
- **Yearly Projection (52 weeks)**: 
  -

('\n\n### การวิเคราะห์ร้านอาหารไทยแถวลุมพินี เซ็นเตอร์ ลาดพร้าว\n\n#### 1. สรุป\n- **ภาพรวม**: การเปิดร้านอาหารไทยในพื้นที่ลุมพินี เซ็นเตอร์ ลาดพร้าวมีศักยภาพสูง เนื่องจากมีประชากรหนาแน่นและมีร้านอาหารไทยที่มีคะแนนดีอยู่หลายแห่งในบริเวณใกล้เคียง\n- **ข้อเสนอแนะ**: ควรเน้นที่คุณภาพของอาหารและบริการเพื่อดึงดูดลูกค้า\n\n#### 2. แนวคิดธุรกิจ\n- **คำอธิบายผลิตภัณฑ์/บริการ**: ร้านอาหารไทยที่นำเสนอเมนูอาหารไทยดั้งเดิมและอาหารฟิวชั่น\n- **จุดขายที่ไม่เหมือนใคร (USP)**: การใช้วัตถุดิบสดใหม่และสูตรอาหารที่เป็นเอกลักษณ์\n- **ความต้องการของลูกค้า**: ลูกค้าต้องการอาหารที่มีคุณภาพและบริการที่รวดเร็ว\n\n#### 3. การวิเคราะห์การแข่งขัน\n- **ภูมิทัศน์การแข่งขัน**: มีร้านอาหารไทยหลายแห่งในพื้นที่ที่มีคะแนนสูง เช่น "Tam Udon E-San Cuisine" และ "Thai Flavours"\n- **รายการเปรียบเทียบ**:\n  - **The Original**: ราคา 100-300 บาท, คะแนน 4.6\n  - **Tam Udon E-San Cuisine**: ราคา 150-350 บาท, คะแนน 5.0\n  - **Thai Flavours**: ราคา 120-320 บาท, คะแนน 5.0\n\n#### 4. การวิจัยตลาดและสภาพตลาด\n- **ภาพรวมตลาด**: ประชาก

In [ ]:
submitUserMessage("if i it passible to sell rice 10 bath each.", keep_chat_history=True, return_reference=True, verbose=True)

================================== Ai Message ==================================
Name: analyst
Tool Calls:
  restaurant_sale_projection (call_b96FTLaWwMMeU3nNFsab81ec)
 Call ID: call_b96FTLaWwMMeU3nNFsab81ec
  Args:
    input_dict: {'base_price': 10, 'category': 'Rice Bowl'}
================================= Tool Message =================================
Name: restaurant_sale_projection

sale projection of Rice Bowl:
week	number of order	sale(forecast)
1	849	8,489
5	2,252	22,521
52	800	8,003

================================== Ai Message ==================================
Name: analyst

Selling rice at 10 baht each is likely not feasible for a few reasons:

### Sales Projection
- **Week 1**: 849 orders, total sales of 8,489 baht
- **Week 5**: 2,252 orders, total sales of 22,521 baht
- **Yearly Projection (52 weeks)**: 800 orders, total sales of 8,003 baht

### Feasibility Analysis
1. **Low Price Point**: Selling rice at 10 baht is significantly lower than the average market price for s

('\n\n### การวิเคราะห์ร้านอาหารไทยแถวลุมพินี เซ็นเตอร์ ลาดพร้าว\n\n#### 1. สรุป\n- **ภาพรวม**: การเปิดร้านอาหารไทยในพื้นที่ลุมพินี เซ็นเตอร์ ลาดพร้าวมีศักยภาพสูง เนื่องจากมีประชากรหนาแน่นและมีร้านอาหารไทยที่มีคะแนนดีอยู่หลายแห่งในบริเวณใกล้เคียง\n- **ข้อเสนอแนะ**: ควรเน้นที่คุณภาพของอาหารและบริการเพื่อดึงดูดลูกค้า\n\n#### 2. แนวคิดธุรกิจ\n- **คำอธิบายผลิตภัณฑ์/บริการ**: ร้านอาหารไทยที่นำเสนอเมนูอาหารไทยดั้งเดิมและอาหารฟิวชั่น\n- **จุดขายที่ไม่เหมือนใคร (USP)**: การใช้วัตถุดิบสดใหม่และสูตรอาหารที่เป็นเอกลักษณ์\n- **ความต้องการของลูกค้า**: ลูกค้าต้องการอาหารที่มีคุณภาพและบริการที่รวดเร็ว\n\n#### 3. การวิเคราะห์การแข่งขัน\n- **ภูมิทัศน์การแข่งขัน**: มีร้านอาหารไทยหลายแห่งในพื้นที่ที่มีคะแนนสูง เช่น "The Original" (4.6), "Tam Udon E-San Cuisine" (5.0), และ "Thai Flavours" (5.0)\n- **รายการเปรียบเทียบ**:\n  - **The Original**: ราคา 100-300 บาท, คะแนน 4.6\n  - **Tam Udon E-San Cuisine**: ราคา 150-350 บาท, คะแนน 5.0\n  - **Thai Flavours**: ราคา 120-320 บาท, คะแนน 5.0\n\n#### 4. การวิจัยตลาดแล